In [1]:
import json
import random

def read_file_lines(filename):
    try:
        with open(filename, "r") as file:
            lines = file.readlines()
            # Stripping newline characters from the end of each line
            lines = [line.strip() for line in lines]
            return lines
    except FileNotFoundError:
        print("File not found.")
        return []


def write_list_to_file(file_path, text_list):
    with open(file_path, "w", encoding="utf-8") as file:
        for item in text_list:
            file.write(str(item) + "\n")


def take_random_elements(input_list, num_elements):
    if num_elements > len(input_list):
        print("Error: Number of elements to take exceeds the length of the list.")
        return []
    else:
        random_elements = random.sample(input_list, num_elements)
        return random_elements


def write_jsonl(list_of_dicts, filename):
    try:
        # Open a file for writing
        with open(filename, "w") as f:
            # Iterate over the list of dictionaries
            for item in list_of_dicts:
                # Convert each dictionary to a JSON string and write it to the file
                json_string = json.dumps(item)
                f.write(json_string + "\n")
        print(f"Data written to {filename} successfully.")
    except Exception as e:
        print(f"Error occurred while writing to {filename}: {e}")


def append_dict_to_jsonl(file_path, dictionary):
    """
    Appends a dictionary as a new JSON object to an existing .jsonl file.
    If the file doesn't exist, it creates a new file.

    Args:
        dictionary (dict): The dictionary to be appended as a JSON object.
        file_path (str): The path to the .jsonl file.
    """
    try:
        with open(file_path, "a", encoding="utf-8") as file:
            json_object = json.dumps(dictionary)
            file.write(json_object + "\n")
    except Exception as e:
        print(f"An error occurred: {e}")

def read_jsonl_file(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data_dict = json.loads(line)
            data_dict.pop("input")
            data_dict.pop("output")
            data_dict.pop("annotations")
            data_list.append(data_dict)
    return data_list

def batch_iterator(iterable, batch_size=10):
    """
    Iterate through an iterable in batches of a specified size.
    
    Args:
        iterable (list, tuple, or any iterable): The iterable to be batched.
        batch_size (int, optional): The size of each batch. Defaults to 10.
        
    Yields:
        list: A batch of items from the iterable.
    """
    iterator = iter(iterable)
    batch = []
    
    try:
        while True:
            for _ in range(batch_size):
                batch.append(next(iterator))
            yield batch
            batch = []
    except StopIteration:
        if batch:
            yield batch

In [2]:
from factscore.factscorer import FactScorer

fs = FactScorer()

[nltk_data] Downloading package punkt to /home/haznitrama/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = []
with open("data/Llama-1-7B-facts.jsonl") as f:
    for line in f:
        dp = json.loads(line)
        data.append(dp)

In [4]:
import numpy as np

for batch_data in batch_iterator(data, batch_size=10):
    topics, generations, atomic_facts = [], [], []
    for dp in batch_data:
        topics.append(dp["topic"])
        generations.append(dp["output"])
        atomic_facts.append(
            [
                atom["text"]
                for sent in dp["annotations"]
                for atom in sent["model-atomic-facts"]
            ]
        )
    out = fs.get_score(
        topics=topics,
        generations=generations,
        atomic_facts=atomic_facts,
        verbose=True,
    )
    
    assert len(out["decisions"]) == len(batch_data)
    
    for idx, decision in enumerate(out["decisions"]):
        s = batch_data[idx].copy()
        count_atoms = 0
        for annotation in s["annotations"]:
            for atom in annotation["model-atomic-facts"]:
                atom["is_supported"] = decision[count_atoms]
        s["factscore"] = np.mean([d["is_supported"] for d in decision])
        append_dict_to_jsonl("Llama-1-7B-factscore.jsonl", s)

CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.00 for 0 words and 0 tokens
100%|██████████| 10/10 [00:00<00:00, 3418.06it/s]


CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.00 for 0 words and 0 tokens
100%|██████████| 10/10 [00:00<00:00, 2193.90it/s]
CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.13 for 195128 words and 260170 tokens
100%|██████████| 10/10 [01:53<00:00, 11.31s/it]
CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.13 for 201143 words and 268190 tokens
100%|██████████| 10/10 [02:38<00:00, 15.81s/it]
CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.11 for 164866 words and 219821 tokens
100%|██████████| 10/10 [02:06<00:00, 12.66s/it]
CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.04 for 62370 words and 83160 tokens
100%|██████████| 10/10 [02:14<00:00, 13.44s/it]
CRITICAL:root:Estimated OpenAI API cost for factscore evaluation ($0.001 per 1000 tokens): $0.07 for 99190 wo

In [ ]:


tot = 0
topics, generations, atomic_facts = [], [], []
with open("data/Llama-1-7B-facts.jsonl") as f:
    count = 0
    for line in f:
        dp = json.loads(line)
        tot += 1
        assert (
            "annotations" in dp
        ), "You can specify `--use_atomic_facts` only when atomic facts are available in the input data already."
        if dp["annotations"] is None:
            continue
        topics.append(dp["topic"])
        generations.append(dp["output"])
        atomic_facts.append(
            [
                atom["text"]
                for sent in dp["annotations"]
                for atom in sent["model-atomic-facts"]
            ]
        )
        count += 1
        # if count >= 10:
        #     break

# print(out)

In [ ]:
out = fs.get_score(
    topics=topics,
    generations=generations,
    atomic_facts=atomic_facts,
    verbose=True,
)

In [6]:
with open(f"sample.json", "w") as f:
    f.write(json.dumps(out) + "\n")